# Interacting with Assets in VAM API

This notebook provides a comprehensive guide on how to interact with different types of assets using the VAM API.
We will cover:
- Querying different asset types
- Filtering assets by execution venue
- Working with different asset types like spot, futures, and equity
- Performing batch operations

For this example, we will use the following assets:
- **Binance Spot**: `BTCUSDT`, `ETHUSDT`
- **Binance Futures**: `BTCUSDT`, `ETHUSDT`
- **Alpaca Equity**: `NVDA`, `MSFT`

We will demonstrate how to query assets across these execution venues.


In [1]:

import dotenv
dotenv.load_dotenv('../../../.env')
import mainsequence    
from mainsequence.vam_client import Asset, AssetFutureUSDM, CurrencyPair
from mainsequence.vam_client.models_binance import BinanceAsset, BinanceAssetFutureUSDM, BinanceCurrencyPair
from mainsequence.vam_client.models_alpaca import AlpacaAsset
from mainsequence.vam_client import CONSTANTS

# Define execution venue symbols
BINANCE_EV = CONSTANTS.BINANCE_EV_SYMBOL
BINANCE_FUTURES_EV = CONSTANTS.BINANCE_FUTURES_EV_SYMBOL
ALPACA_EV = CONSTANTS.ALPACA_EV_SYMBOL

## Querying All Assets

To get all assets available in the system, we can use the `Asset.filter()` method.


In [2]:

# Query all assets
all_assets= Asset.filter()
print(f"Total Assets: {len(all_assets)}")


2025-02-27T11:04:28.191670Z [error    ] Error connecting http://192.168.178.69:8010/orm/api/asset/?limit=100&offset=5100  application_name=ms-sdk data_source_id=28 job_run_id=None project_id=16
Traceback (most recent call last):
  File "/home/jose/code/MainSequenceClientSide/mainsequence-sdk/venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "/home/jose/code/MainSequenceClientSide/mainsequence-sdk/venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 468, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/jose/code/MainSequenceClientSide/mainsequence-sdk/venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 463, in _make_request
    httplib_response = conn.getresponse()
  File "/home/jose/.pyenv/versions/3.9.19/lib/python3.9/http/client.py", line 1377, in getresponse
    response.begin()
  File "/home/jose/.pyenv/versions/3.9.19/lib/p

Total Assets: 15615


## Querying Assets by Execution Venue

Assets belong to different execution venues. We can filter assets based on where they are listed.


In [3]:
# Query Binance Spot assets
binance_assets = Asset.filter(execution_venue__symbol=BINANCE_EV)
print(f"Total Binance Spot Assets: {len(binance_assets)}")

# Query Binance Futures assets
binance_futures_assets = Asset.filter(execution_venue__symbol=BINANCE_FUTURES_EV)
print(f"Total Binance Futures Assets: {len(binance_futures_assets)}")

# Query Alpaca Equity assets
alpaca_assets= Asset.filter(execution_venue__symbol=ALPACA_EV)
print(f"Total Alpaca Equity Assets: {len(alpaca_assets)}")


Total Binance Spot Assets: 3673
Total Binance Futures Assets: 421
Total Alpaca Equity Assets: 11477


## Filtering Assets by Type

Assets can be classified into different types such as spot, futures, and equity.


In [4]:
# Filtering Binance Spot Crypto Assets
spot_crypto_assets = BinanceAsset.filter(asset_type=CONSTANTS.ASSET_TYPE_CRYPTO_SPOT)
print(f"Total Spot Crypto Assets on Binance: {len(spot_crypto_assets)}")

# Filtering Binance Futures USDM Assets
futures_assets= BinanceAssetFutureUSDM.filter(asset_type=CONSTANTS.ASSET_TYPE_CRYPTO_USDM)
print(f"Total Futures Assets on Binance: {len(futures_assets)}")

# Filtering Alpaca Cash Equity Assets
cash_equity_assets = AlpacaAsset.filter(asset_type=CONSTANTS.ASSET_TYPE_CASH_EQUITY)
print(f"Total Cash Equity Assets on Alpaca: {len(cash_equity_assets)}")


Total Spot Crypto Assets on Binance: 705
Total Futures Assets on Binance: 413


2025-02-27T11:08:01.829881Z [error    ] Error connecting http://192.168.178.69:8010/orm/api/alpaca/asset/spot/  application_name=ms-sdk data_source_id=28 job_run_id=None project_id=16
Traceback (most recent call last):
  File "/home/jose/code/MainSequenceClientSide/mainsequence-sdk/venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "/home/jose/code/MainSequenceClientSide/mainsequence-sdk/venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 468, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/home/jose/code/MainSequenceClientSide/mainsequence-sdk/venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 463, in _make_request
    httplib_response = conn.getresponse()
  File "/home/jose/.pyenv/versions/3.9.19/lib/python3.9/http/client.py", line 1377, in getresponse
    response.begin()
  File "/home/jose/.pyenv/versions/3.9.19/lib/python3.9/h

Total Cash Equity Assets on Alpaca: 11390


## Querying Specific Assets

To retrieve specific assets, we can filter by their symbols.


In [5]:
# Query specific Binance Spot assets
btc_usdt_spot, _ = BinanceAsset.filter(symbol="BTCUSDT")
eth_usdt_spot, _ = BinanceAsset.filter(symbol="ETHUSDT")
print(f"BTCUSDT Spot Asset: {btc_usdt_spot}")
print(f"ETHUSDT Spot Asset: {eth_usdt_spot}")

# Query specific Binance Futures assets
btc_usdt_futures, _ = BinanceAssetFutureUSDM.filter(symbol="BTCUSDT")
eth_usdt_futures, _ = BinanceAssetFutureUSDM.filter(symbol="ETHUSDT")
print(f"BTCUSDT Futures Asset: {btc_usdt_futures}")
print(f"ETHUSDT Futures Asset: {eth_usdt_futures}")

# Query specific Alpaca equity assets
nvda_equity, _ = AlpacaAsset.filter(symbol="NVDA")
msft_equity, _ = AlpacaAsset.filter(symbol="MSFT")
print(f"NVDA Equity Asset: {nvda_equity}")
print(f"MSFT Equity Asset: {msft_equity}")


ValueError: not enough values to unpack (expected 2, got 0)

## Using `filter_with_asset_class`

The `filter_with_asset_class` method is an extended version of the `filter` method.
It ensures that each returned asset is an instance of its correct class based on its `asset_type`.

### When to Use?
- When querying multiple asset types and needing them returned with the correct class.
- When working with assets across execution venues and requiring proper type differentiation.

### Example 1: Query Binance Spot Assets with Correct Classes


In [9]:
# Query Binance Spot assets with correct asset classes
binance_assets = Asset.filter_with_asset_class(execution_venue__symbol=CONSTANTS.BINANCE_EV_SYMBOL)

# Displaying asset information
for asset in binance_assets:
    print(f"Asset: {asset.symbol}, Type: {asset.asset_type}, Class: {type(asset).__name__}")


ValidationError: 2 validation errors for BinanceCurrencyPair
base_asset
  Field required [type=missing, input_value={'id': 11489, 'symbol': '...'unique_symbol': 'LINK'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
quote_asset
  Field required [type=missing, input_value={'id': 11489, 'symbol': '...'unique_symbol': 'LINK'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

### Example 2: Query Binance Futures and Alpaca Equity Assets

This example demonstrates how the method ensures each asset is returned with its appropriate class.


In [ ]:
# Query Binance Futures and Alpaca Equity Assets with correct classes
binance_futures_assets = Asset.filter_with_asset_class(execution_venue__symbol=CONSTANTS.BINANCE_FUTURES_EV_SYMBOL)
alpaca_equity_assets = Asset.filter_with_asset_class(execution_venue__symbol=CONSTANTS.ALPACA_EV_SYMBOL)

# Displaying results
for asset in binance_futures_assets + alpaca_equity_assets:
    print(f"Asset: {asset.symbol}, Type: {asset.asset_type}, Class: {type(asset).__name__}")